<a href="https://colab.research.google.com/github/ayyucedemirbas/Retrieval-Augmented_Generation/blob/main/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community pypdf sentence_transformers faiss-cpu langchain-anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.0/894.0 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from google.colab import userdata

In [3]:
document_url = "https://arxiv.org/pdf/2306.00890.pdf"
loader = PyPDFLoader(document_url)
pages = loader.load()

In [4]:
print(pages[0].page_content[0:250])

LLaV A-Med: Training a Large Language-and-Vision
Assistant for Biomedicine in One Day
Chunyuan Li∗, Cliff Wong∗, Sheng Zhang∗, Naoto Usuyama, Haotian Liu, Jianwei Yang
Tristan Naumann, Hoifung Poon, Jianfeng Gao
Microsoft
https://aka.ms/llava-med
Abs


In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    length_function=len,
    is_separator_regex=False,
)
chunks = text_splitter.split_documents(pages)
print(chunks[0])

page_content='LLaV A-Med: Training a Large Language-and-Vision
Assistant for Biomedicine in One Day
Chunyuan Li∗, Cliff Wong∗, Sheng Zhang∗, Naoto Usuyama, Haotian Liu, Jianwei Yang
Tristan Naumann, Hoifung Poon, Jianfeng Gao
Microsoft
https://aka.ms/llava-med
Abstract
Conversational generative AI has demonstrated remarkable promise for empow-' metadata={'source': 'https://arxiv.org/pdf/2306.00890.pdf', 'page': 0}


In [6]:
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

chunk_texts = list(map(lambda d: d.page_content, chunks))
embeddings = bge_embeddings.embed_documents(chunk_texts)
print(embeddings[0])

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[-0.05840783566236496, 0.0018189228139817715, 0.0011776224710047245, -0.027336575090885162, 0.011472324840724468, -0.014415203593671322, 0.01403537206351757, 0.03533104434609413, 0.02239002287387848, -0.0036270231939852238, -0.022023219615221024, -0.040414392948150635, 0.037284959107637405, 0.02262328937649727, 0.05287672579288483, 0.016154736280441284, 0.007292558904737234, -0.014497536234557629, -0.017463577911257744, 0.03464886546134949, 0.0035230882931500673, -0.031132366508245468, 0.0012668234994634986, -0.03430065140128136, -0.045267265290021896, -0.004855607636272907, -0.021141279488801956, -0.05100385099649429, 0.013304227031767368, -0.20457246899604797, 0.00478530116379261, 0.021882683038711548, 0.07344349473714828, 0.0018850117921829224, -0.032800838351249695, 0.03250788524746895, -0.014656667597591877, 0.04177973419427872, -0.05093768611550331, 0.03418421372771263, -0.007706900127232075, -0.0058563887141644955, -0.014052328653633595, -0.05922034755349159, -0.0098795657977461

In [7]:
text_embedding_pairs = zip(chunk_texts, embeddings)
db = FAISS.from_embeddings(text_embedding_pairs, bge_embeddings)

In [8]:
query = "Explain the LLaVA-Med model architecture."
contexts = db.similarity_search(query, k=5)
print(contexts[0])

page_content='collected in Section 3. As demonstrated in the experiments to be described later, the LLaV A-Med
model at this stage is able to not only be served as a biomedical visual assistant to interact with
users, but also achieve good zero-shot task transfer performance when evaluated on well-established
biomedical VQA datasets.'


In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert at answering questions based on a context extracted from a document. The context extracted from the document is: {context}"),
        ("human", "{question}"),
    ]
)

api_key = userdata.get('key') #https://console.anthropic.com/dashboard
model = ChatAnthropic(model='claude-3-haiku-20240307', api_key=api_key)

chain = prompt | model

response = chain.invoke({
    "context": '\n\n'.join(list(map(lambda c: c.page_content, contexts))),
    "question": query
})

In [ ]:
response.content